## PReader (old)
I couldn't make the PReader class be just a recursive algorithm. For long strings, this would mean that the stack would get very big. Some sort of buffer would have to prevent the stack from getting too big while also keeping the right order of characters. 

I created a char array from the string and a cursor. The cursor is a position indicator for which character is being read. In addition we would have to check for certain trigger characters, like operators for starting a new operation, parentheses to capture the operation in, and letters that would be variables. Taking this into account, we should also create a stack for read operations up until the current character. We would then interpret the operator when a closing parenthesis occurs.

## Testing arena below

In [1]:
from proposition import *
from proposition_operator import *
from proposition_parsing import *
from proposition_export import *
from hashing import *
from truthtable import *
from debugger import *

and_operator = OperatorFactory[AndOperator]
or_operator = OperatorFactory[OrOperator]
impl_operator = OperatorFactory[ImplicationOperator]
biimpl_operator = OperatorFactory[BiimplicationOperator]
not_operator = OperatorFactory[NotOperator]

prop = CompoundProposition(or_operator,
    CompoundProposition(or_operator, Variable('A'), Variable('B')),
    Variable('C')
)

print(prop)
print(prop.infix())

|(|(A,B),C)
((A ∨ B) ∨ C)


In [2]:
truthtable = TruthTable(prop)

print(truthtable)

A B C result
0 0 0 0
0 0 1 1
0 1 0 1
0 1 1 1
1 0 0 1
1 0 1 1
1 1 0 1
1 1 1 1



In [3]:
simplified_truthtable = truthtable.simplify()

print(simplified_truthtable)

A B C result
0 0 0 0
* * 1 1
* 1 * 1
1 * * 1



In [4]:
print(simplified_truthtable.get_binary_string())
print(simplified_truthtable.get_hash())

1110
E


In [5]:
prop = PropositionParser("&(~(A),>(>(C,|(D,E)),~(B)))").read()
print(prop.infix())

truthtable = TruthTable(prop)
print(truthtable)

(¬A ∧ ((C => (D ∨ E)) => ¬B))
A B C D E result
0 0 0 0 0 1
0 0 0 0 1 1
0 0 0 1 0 1
0 0 0 1 1 1
0 0 1 0 0 1
0 0 1 0 1 1
0 0 1 1 0 1
0 0 1 1 1 1
0 1 0 0 0 0
0 1 0 0 1 0
0 1 0 1 0 0
0 1 0 1 1 0
0 1 1 0 0 1
0 1 1 0 1 0
0 1 1 1 0 0
0 1 1 1 1 0
1 0 0 0 0 0
1 0 0 0 1 0
1 0 0 1 0 0
1 0 0 1 1 0
1 0 1 0 0 0
1 0 1 0 1 0
1 0 1 1 0 0
1 0 1 1 1 0
1 1 0 0 0 0
1 1 0 0 1 0
1 1 0 1 0 0
1 1 0 1 1 0
1 1 1 0 0 0
1 1 1 0 1 0
1 1 1 1 0 0
1 1 1 1 1 0



In [6]:
truthtable.simplify()
print(truthtable)
print(f"hash: {truthtable.get_hash()}")

A B C D E result
0 0 0 0 0 1
0 0 0 0 1 1
0 0 0 1 0 1
0 0 0 1 1 1
0 0 1 0 0 1
0 0 1 0 1 1
0 0 1 1 0 1
0 0 1 1 1 1
0 1 0 0 0 0
0 1 0 0 1 0
0 1 0 1 0 0
0 1 0 1 1 0
0 1 1 0 0 1
0 1 1 0 1 0
0 1 1 1 0 0
0 1 1 1 1 0
1 * * * * 0

hash: 010FF


In [7]:
dnf_prop = truthtable.dnf()
print(dnf_prop.infix())

dnf_truthtable = TruthTable(dnf_prop)
dnf_truthtable.simplify()
print(dnf_truthtable)
print(f"hash: {dnf_truthtable.get_hash()}")

((¬A ∧ ¬B ∧ ¬C ∧ ¬D ∧ ¬E) ∨ (¬A ∧ ¬B ∧ ¬C ∧ ¬D ∧ E) ∨ (¬A ∧ ¬B ∧ ¬C ∧ D ∧ ¬E) ∨ (¬A ∧ ¬B ∧ ¬C ∧ D ∧ E) ∨ (¬A ∧ ¬B ∧ C ∧ ¬D ∧ ¬E) ∨ (¬A ∧ ¬B ∧ C ∧ ¬D ∧ E) ∨ (¬A ∧ ¬B ∧ C ∧ D ∧ ¬E) ∨ (¬A ∧ ¬B ∧ C ∧ D ∧ E) ∨ (¬A ∧ B ∧ C ∧ ¬D ∧ ¬E))
A B C D E result
0 0 0 0 0 1
0 0 0 0 1 1
0 0 0 1 0 1
0 0 0 1 1 1
0 0 1 0 0 1
0 0 1 0 1 1
0 0 1 1 0 1
0 0 1 1 1 1
0 1 0 0 0 0
0 1 0 0 1 0
0 1 0 1 0 0
0 1 0 1 1 0
0 1 1 0 0 1
0 1 1 0 1 0
0 1 1 1 0 0
0 1 1 1 1 0
1 * * * * 0

hash: 010FF


# Bigger test cases

In [8]:
def print_details(prop1, prop2):
    tt1 = TruthTable(prop1)
    tt2 = TruthTable(prop2)

    print(prop1.infix())
    print(tt1)
    print("hash: " + tt1.get_hash())
    print(prop2.infix())
    print(tt2)
    print("hash: " + tt2.get_hash())

In [9]:
# prop = PropositionParser("~(~(=(~(~(&(A,C))),B)))").read()
prop = PropositionParser("=(&(A,C),B)").read()

tfunc = lambda x : ()
analysis_hash = lambda prop : print(f"analysis - hash: {TruthTable(prop).get_hash()} - infix: {prop.infix()}")
analyzing_functions = [analysis_hash]
print_details(prop, prop.cnf(debugger=Debugger(trace_function=tfunc, analyzing_functions=analyzing_functions))) # I have been trying to fix this but I cant get 100% correctness

analysis - hash: 7 - infix: (¬A ∨ ¬C)
analysis - hash: 7 - infix: (¬A ∨ ¬C)
analysis - hash: 7 - infix: (¬A ∨ ¬C)
analysis - hash: 7 - infix: (¬A ∨ ¬C)
analysis - hash: 01 - infix: (¬A ∧ ¬C ∧ ¬B)
analysis - hash: 8 - infix: (A ∧ C)
analysis - hash: 80 - infix: (A ∧ C ∧ B)
analysis - hash: 01 - infix: (¬A ∧ ¬C ∧ ¬B)
analysis - hash: 1 - infix: (¬C ∧ ¬B)
analysis - hash: 81 - infix: (((A ∧ C) ∧ B) ∨ (¬A ∧ ¬C ∧ ¬B))
analysis - hash: 91 - infix: ((A ∨ ¬C) ∧ (A ∨ ¬B) ∧ (C ∨ ¬C) ∧ (C ∨ ¬B) ∧ (B ∨ ¬C) ∧ (B ∨ ¬B))
analysis - hash: 81 - infix: (((A ∧ C) ∧ B) ∨ (¬A ∧ ¬C ∧ ¬B))
analysis - hash: 91 - infix: ((A ∨ ¬C) ∧ (A ∨ ¬B) ∧ (C ∨ ¬C) ∧ (C ∨ ¬B) ∧ (B ∨ ¬C) ∧ (B ∨ ¬B))
((A ∧ C) <=> B)
A B C result
0 0 0 1
0 0 1 1
0 1 0 0
0 1 1 0
1 0 0 1
1 0 1 0
1 1 0 0
1 1 1 1

hash: 93
((A ∨ ¬C) ∧ (A ∨ ¬B) ∧ (C ∨ ¬C) ∧ (C ∨ ¬B) ∧ (B ∨ ¬C) ∧ (B ∨ ¬B))
A B C result
0 0 0 1
0 0 1 0
0 1 0 0
0 1 1 0
1 0 0 1
1 0 1 0
1 1 0 0
1 1 1 1

hash: 91


In [10]:
print_details(prop, PropositionParser("|(&(&(A,C),B),&(~(&(A,C)),~(B)))").read())

((A ∧ C) <=> B)
A B C result
0 0 0 1
0 0 1 1
0 1 0 0
0 1 1 0
1 0 0 1
1 0 1 0
1 1 0 0
1 1 1 1

hash: 93
(((A ∧ C) ∧ B) ∨ (¬(A ∧ C) ∧ ¬B))
A B C result
0 0 0 1
0 0 1 1
0 1 0 0
0 1 1 0
1 0 0 1
1 0 1 0
1 1 0 0
1 1 1 1

hash: 93
